In [2]:
import pandas as pd
import gtfs_kit
import folium
import seaborn

In [3]:
# load gtfs feed into pandas dataframes
feed = gtfs_kit.read_feed("gtfs_fp2024_2023-10-18_04-15.zip", dist_units="km")

In [16]:
feed.stops

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,1100723,"Inzlingen, Zoll",47.585586,7.672664,NaN,NaN
1,1101316,"Büsingen, Bürgerhaus",47.696277,8.688029,NaN,NaN
2,1101319,"Büsingen, Kirche St. Michael",47.698780,8.703058,NaN,NaN
3,1101322,Büsingen (Schifflände),47.695430,8.693320,NaN,NaN
4,1101323,"Büsingen, Schifflände",47.696875,8.693877,NaN,NaN
...,...,...,...,...,...,...
33133,Parent8769158,Arcades,46.284504,6.085844,1.0,NaN
33134,Parent8769399,Place d'Allemogne,46.244454,5.983822,1.0,NaN
33135,Parent8771695,Porte de France,46.240962,6.028792,1.0,NaN
33136,Parent8774549,Annemasse,46.199344,6.236401,1.0,NaN


In [4]:
feed.stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,1.TA.91-10-A-j24-1.1.H,19:37:00,19:37:00,8503054:0:1,1,0,0
1,1.TA.91-10-A-j24-1.1.H,19:38:00,19:38:00,8503053:0:1,2,0,0
2,1.TA.91-10-A-j24-1.1.H,19:39:00,19:39:00,8503052:0:1,3,0,0
3,1.TA.91-10-A-j24-1.1.H,19:41:00,19:42:00,8503051:0:1,4,0,0
4,1.TA.91-10-A-j24-1.1.H,19:44:00,19:44:00,8503090:0:1,5,0,0
...,...,...,...,...,...,...,...
6743194,99.TA.96-923-j24-1.8.R,15:17:00,15:17:00,8580743,8,0,0
6743195,99.TA.96-923-j24-1.8.R,15:18:00,15:18:00,8571319,9,0,0
6743196,99.TA.96-923-j24-1.8.R,15:18:00,15:18:00,8585080,10,0,0
6743197,99.TA.96-923-j24-1.8.R,15:19:00,15:19:00,8580370,11,0,0


In [6]:
feed.trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id
0,91-10-A-j24-1,TA+vlr60,1.TA.91-10-A-j24-1.1.H,Zürich HB SZU,12920,0,NaN
1,91-10-A-j24-1,TA+vlr60,10.TA.91-10-A-j24-1.1.H,Zürich HB SZU,12902,0,NaN
2,91-10-A-j24-1,TA+vlr60,100.TA.91-10-A-j24-1.7.H,Zürich HB SZU,12838,0,NaN
3,91-10-A-j24-1,TA+ner60,101.TA.91-10-A-j24-1.7.H,Zürich HB SZU,12948,0,NaN
4,91-10-A-j24-1,TA+ner60,102.TA.91-10-A-j24-1.7.H,Zürich HB SZU,12932,0,NaN
...,...,...,...,...,...,...,...
760319,96-990-j24-1,TA+v1t60,5.TA.96-990-j24-1.2.H,Grosse Scheidegg,601,0,NaN
760320,96-990-j24-1,TA+v1t60,6.TA.96-990-j24-1.3.R,Schwarzwaldalp,706,1,NaN
760321,96-990-j24-1,TA+v1t60,7.TA.96-990-j24-1.3.R,Schwarzwaldalp,602,1,NaN
760322,96-990-j24-1,TA+v1t60,8.TA.96-990-j24-1.3.R,Schwarzwaldalp,704,1,NaN


In [5]:
feed.routes

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
0,91-10-A-j24-1,78,S10,NaN,S,109
1,91-10-B-j24-1,11,S10,NaN,S,109
2,91-10-C-j24-1,65,S10,NaN,S,109
3,91-10-E-j24-1,3849,10,NaN,T,900
4,91-10-j24-1,37,10,NaN,T,900
...,...,...,...,...,...,...
3621,96-948-j24-1,801,66,NaN,B,700
3622,96-953-j24-1,801,184,NaN,B,700
3623,96-954-j24-1,801,185,NaN,B,700
3624,96-987-j24-1,801,143,NaN,B,700


In [3]:
# only consider parent stations when searching for destinations, because the platforms dont matter
# this makes it more or less useless for last kilometer transportation (which is fine, for now)
every_station = feed .stops.loc[feed.stops.stop_id.str.contains("Parent")]
every_station.loc[every_station.stop_name.str.contains("singen", case=False)]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
30785,Parent8014407,Norsingen,47.939249,7.726194,1.0,NaN
30830,Parent8014533,Geisingen,47.921343,8.653947,1.0,NaN
30835,Parent8014558,Singen (Hohentwiel),47.758437,8.840384,1.0,NaN
30886,Parent8029338,Mössingen,48.404309,9.046529,1.0,NaN
30888,Parent8029343,Bisingen,48.312769,8.921034,1.0,NaN
30895,Parent8029434,Trossingen Bahnhof,48.087662,8.585154,1.0,NaN
30908,Parent8063365,Singen (Htw)-Landesgartenschau,47.760738,8.827044,1.0,NaN
30922,Parent8099985,Singen (Htw) Industriegebiet,47.759451,8.872130,1.0,NaN
30986,Parent8500212,Oensingen,47.285036,7.709872,1.0,NaN
31213,Parent8502105,Othmarsingen,47.407433,8.214806,1.0,NaN


In [11]:
every_station.loc[every_station["stop_id"] == "Parent8014533"]["stop_name"].iloc[0]

'Geisingen'

In [12]:
# this should be done over an selector from data above
# station = every_station.loc[every_station.stop_id.str.contains("Rotkreuz", case=False)]
station = every_station.loc[every_station.stop_id.str.contains("Parent8500212", case=False)]

manual_exclusions = ["IC21"]

In [13]:
# get all platforms
platforms = feed.stops.loc[feed.stops["parent_station"] == station["stop_id"].values[0]]

In [14]:
all_stop_times = feed.stop_times.loc[feed.stop_times["stop_id"].isin(platforms["stop_id"].values)]

In [10]:
feed.trips.loc[feed.trips["trip_id"].isin(all_stop_times["trip_id"].unique())]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id
3625,91-11-L-j24-1,TA+mxq60,1.TA.91-11-L-j24-1.1.H,Oensingen,9410,0,NaN
3626,91-11-L-j24-1,TA,10.TA.91-11-L-j24-1.5.H,Langenthal,438,0,142
3628,91-11-L-j24-1,TA+dxq60,101.TA.91-11-L-j24-1.10.R,Oensingen,281,1,NaN
3629,91-11-L-j24-1,TA+mxq60,102.TA.91-11-L-j24-1.11.R,Oensingen,413,1,179
3630,91-11-L-j24-1,TA+qxq60,103.TA.91-11-L-j24-1.11.R,Oensingen,413,1,180
...,...,...,...,...,...,...,...
37243,91-5-A-j24-1,TA,97.TA.91-5-A-j24-1.429.R,Lausanne,1522,1,NaN
37244,91-5-A-j24-1,TA,98.TA.91-5-A-j24-1.429.R,Lausanne,1532,1,NaN
37245,91-5-A-j24-1,TA+zwq60,99.TA.91-5-A-j24-1.429.R,Lausanne,1512,1,NaN
60649,91-E7-Y-j24-1,TA+9jr60,1.TA.91-E7-Y-j24-1.1.H,Biel/Bienne,4544,0,NaN


In [15]:
route_ids = feed.trips.loc[feed.trips["trip_id"].isin(all_stop_times["trip_id"].unique())]["route_id"].unique()

# -Y are artificial routes
# excludes too many...
# route_ids = list(filter(lambda id: "-Y" not in id, all_routes))
# routes = feed.routes.loc[feed.routes["route_id"].isin(route_ids)]


all_routes = feed.routes.loc[feed.routes["route_id"].isin(route_ids)]

routes = all_routes.loc[all_routes["route_short_name"] != "EXT"]
routes
# there are broken trips and routes which claim to stop but actually dont

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
11,91-11-L-j24-1,81,S11,NaN,S,109
94,91-20-A-j24-1,11,S20,NaN,S,109
113,91-22-j24-1,11,S22,NaN,S,109
326,91-5-A-j24-1,11,IC5,NaN,IC,102
542,91-E7-Y-j24-1,11,RE,NaN,RE,106


In [12]:
with_wrong_route = feed.trips.loc[feed.trips["trip_id"].isin(all_stop_times["trip_id"].unique())]
wrong_trips = with_wrong_route.loc[with_wrong_route["route_id"] == "91-21-D-j23-1"]
wrong_stops = feed.stop_times.loc[feed.stop_times["trip_id"].isin(wrong_trips["trip_id"].values)]
wrong_stops.loc[wrong_stops["stop_id"].isin(platforms["stop_id"].values)]
len(wrong_trips)
wrong_stops.loc[wrong_stops["stop_id"].isin(platforms["stop_id"].values)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type


In [16]:
route_trips = feed.trips.loc[feed.trips["route_id"].isin(route_ids)]

# all stops of the trips
relevant_stops = feed.stop_times.loc[
    feed.stop_times["trip_id"].isin(route_trips["trip_id"].values)
]

# parse arrival and departure to timedeltas
relevant_stops.loc[:, "arrival_time_parsed"] = pd.to_timedelta(
    relevant_stops["arrival_time"]
)
relevant_stops.loc[:, "departure_time_parsed"] = pd.to_timedelta(
    relevant_stops["departure_time"]
)

# limit arrival and departure to "normal" times 06-22
# pickup, dropoff type not 0 means no normal passenger transfer
filtered_stops = relevant_stops.loc[
    ((relevant_stops["pickup_type"] == 0) & (relevant_stops["drop_off_type"] == 0))
    & (
        (
            (relevant_stops["arrival_time_parsed"] > pd.Timedelta(6, unit="h"))
            & (relevant_stops["arrival_time_parsed"] < pd.Timedelta(22, unit="h"))
        )
        | (
            (relevant_stops["departure_time_parsed"] > pd.Timedelta(6, unit="h"))
            & (relevant_stops["departure_time_parsed"] < pd.Timedelta(22, unit="h"))
        )
    )
]

merged = pd.merge(route_trips, filtered_stops, on="trip_id")

# find longest trip, so the path generation does not get confused by direction or shorter trips
# might be problematic if the longest is a rare trip and not representative of the usual stops
longest_trips = merged.loc[merged.groupby(["route_id"])["stop_sequence"].idxmax()]
longest_trips_stop_times = feed.stop_times.loc[
    feed.stop_times["trip_id"].isin(longest_trips["trip_id"].values)
]

# find stops to display
longest_trips_parent_stops = feed.stops.loc[
    feed.stops.loc[:, "stop_id"].isin(longest_trips_stop_times["stop_id"].values)
]

# parent stops and stops with no parent
all_stops = pd.concat(
    [
        longest_trips_parent_stops.drop_duplicates(["parent_station"]),
        longest_trips_parent_stops[longest_trips_parent_stops["parent_station"].isna()],
    ]
)


# join all the data into one dataframe
stop_data = pd.merge(longest_trips_stop_times, route_trips, on="trip_id")
stop_data = pd.merge(stop_data, routes, on="route_id")
stop_data = pd.merge(stop_data, longest_trips_parent_stops, on="stop_id")
longest_trips_stop_times.loc[longest_trips_stop_times["trip_id"] == "258.TA.91-21-D-j23-1.76.H"]
relevant_stops.loc[relevant_stops["trip_id"] == "258.TA.91-21-D-j23-1.76.H"]

/tmp/ipykernel_8984/3980140712.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_stops.loc[:, "arrival_time_parsed"] = pd.to_timedelta(
/tmp/ipykernel_8984/3980140712.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_stops.loc[:, "departure_time_parsed"] = pd.to_timedelta(


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,arrival_time_parsed,departure_time_parsed


In [26]:
[0.13777, 0.537492, 0.554906]

[0.13777, 0.537492, 0.554906]

In [23]:
feed.trips.loc[feed.trips["route_id"].isin(routes["route_id"])]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id
3625,91-11-L-j24-1,TA+mxq60,1.TA.91-11-L-j24-1.1.H,Oensingen,9410,0,NaN
3626,91-11-L-j24-1,TA,10.TA.91-11-L-j24-1.5.H,Langenthal,438,0,142
3627,91-11-L-j24-1,TA+qxq60,100.TA.91-11-L-j24-1.9.R,Bannwil,203,1,NaN
3628,91-11-L-j24-1,TA+dxq60,101.TA.91-11-L-j24-1.10.R,Oensingen,281,1,NaN
3629,91-11-L-j24-1,TA+mxq60,102.TA.91-11-L-j24-1.11.R,Oensingen,413,1,179
...,...,...,...,...,...,...,...
37243,91-5-A-j24-1,TA,97.TA.91-5-A-j24-1.429.R,Lausanne,1522,1,NaN
37244,91-5-A-j24-1,TA,98.TA.91-5-A-j24-1.429.R,Lausanne,1532,1,NaN
37245,91-5-A-j24-1,TA+zwq60,99.TA.91-5-A-j24-1.429.R,Lausanne,1512,1,NaN
60649,91-E7-Y-j24-1,TA+9jr60,1.TA.91-E7-Y-j24-1.1.H,Biel/Bienne,4544,0,NaN


Function over beauty, if it needs to be more attractive try drawing the lines next to each other and follow the actual map (download osm track data). 

You create a tool that helps with a particular analytic task or lets the user explore the results of your work
or interact with your model/analysis (e.g. simple classification model, clustering, dimensionality
reduction). The task and target user need to be defined and described in the report. There need to be at
least 2 interactive charts with filtering elements and interactivity between them (the minimum
requirement increases by +1 chart for each additional team member). I want you to focus on providing
interactivity that supports the task and on choosing chart types that support the target user.

In [15]:
map = folium.Map(tiles="cartodbpositron")

# own rough path generation, because there is no shapes data
grouped = stop_data.sort_values(["trip_id", "stop_sequence"]).groupby("trip_id")
colors = seaborn.color_palette("viridis", n_colors=grouped.ngroups).as_hex()
idx = 0
skipped_routes = []
path_layer = folium.FeatureGroup(name="Paths")
for name, group in grouped:
    destination_stop_in_route = False
    stop_coords = []
    tooltip_content = f"{group.iloc[0]['route_short_name']}"

    for row_index, row in group.iterrows():
        stop_coords.append(row[["stop_lat", "stop_lon"]].values)
        tooltip_content += f"<br/>{row['stop_name']}"

        # remove leftover markers or pre condition it
        if (row["parent_station"] == station["stop_id"]).bool():
            destination_stop_in_route = True

    # hide a few more routes
    if destination_stop_in_route:
        # TODO check if there are duplicates, if there are render them side by side
        folium.PolyLine(stop_coords, tooltip=tooltip_content, color=colors[idx]).add_to(path_layer)
        idx += 1
    else:
        skipped_routes.append(name)

path_layer.add_to(map)

marker_layer = folium.FeatureGroup(name="Stops")
# drop duplicates kinda weird, why do some routes get all stops removed
shown_stops = stop_data.drop_duplicates(subset=["parent_station"], keep='first')
# remove routes from map if they arent present here??
shown_stops = shown_stops.loc[~shown_stops["trip_id"].isin(skipped_routes)]

for row in shown_stops.to_dict(orient="records"):
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        popup=row["stop_name"],
        radius=3,
        fill=True,
        color=colors[0],
        weight=0.5,
        fillOpacity= 0.75,
    ).add_to(marker_layer)

marker_layer.add_to(map)


bounds = [
    (stop_data.stop_lat.min(), stop_data.stop_lon.min()),
    (stop_data.stop_lat.max(), stop_data.stop_lon.max()),
]
map.fit_bounds(bounds, padding=[1, 1])
map